In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

In [2]:
import fastbook
fastbook.setup_book()

In [3]:
from fastbook import *
from fastai.vision.widgets import *

In [4]:
tree_types = ['oak', 'silver birch', 'pine', 'beach', 'holly']
path = Path('trees')

In [5]:
if not path.exists():
    path.mkdir()
    for ttype in tree_types:
        dest = path / ttype.replace(' ', '_')
        urls = search_images_ddg(f'{ttype} tree')
        download_images(dest, urls=urls)

In [6]:
fs = get_image_files(path)
failed = verify_images(fs)
print(len(failed))
failed.map(Path.unlink);

0


In [7]:
trees = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.4),
    batch_tfms=aug_transforms(mult=0.5),
)

In [ ]:
dls = trees.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

/Users/tom/projects/fastbook/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: The operator 'aten::_linalg_solve_ex.result' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  ret = func(*args, **kwargs)


In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)